I will be working with fault code data and vehicle onboard diagnostic data to try and predict an upcoming full derate. These are indicated by an SPN 5246.

In [18]:
import pandas as pd
from datetime import datetime
import geopandas as gpd
from geopy.distance import distance

In [19]:
parse_dates=['EventTimeStamp']

In [20]:
faults = pd.read_csv('data/J1939Faults.csv', low_memory=False, parse_dates=['EventTimeStamp'])
service_fault = pd.read_excel('data/ServiceFaultCodes.xlsx')
vehicle_Diagnostics = pd.read_csv('data/VehicleDiagnosticOnboardData.csv')

/Users/alisoncordoba/opt/anaconda3/lib/python3.9/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():


In [21]:
# filter out rows that have more than 5 characters in the 'EquipmentID' column
faults =faults[faults['EquipmentID'].str.len() <= 5]

Basic EDA to check how the data looks like:

In [5]:
#get rid of faultValue and actionDescription since they haven't been filled in. 
#FaultId = RecordID

faults

,RecordID,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,faultValue,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp
0,1,990349,2015-02-21 10:47:13,Low (Severity Low) Engine Coolant Level,NaN,unknown,unknown,unknown,unknown,0,111,17,True,2,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:34:25.000
1,2,990360,2015-02-21 11:34:34,NaN,NaN,unknown,unknown,unknown,unknown,11,629,12,True,127,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:35:10.000
2,3,990364,2015-02-21 11:35:31,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,11,1807,2,False,127,NaN,1369,105336226,41.421250,-87.767361,2015-02-21 11:35:26.000
3,4,990370,2015-02-21 11:35:33,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,11,1807,2,True,127,NaN,1369,105336226,41.421018,-87.767361,2015-02-21 11:36:08.000
4,5,990416,2015-02-21 11:39:41,NaN,NaN,22281684P01*22357957P01*22362082P01*,13063430,0USA13_13_0415_2238A,VOLVO,0,4364,17,False,2,NaN,1674,105427130,38.416481,-89.442638,2015-02-21 11:39:37.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1187330,1248454,123904424,2020-03-06 14:00:26,Low (Severity Low) Catalyst Tank Level,NaN,04384413*22383729*082218154102*60701732*G1*BGT*,80156139,6X1u17D1500000000,CMMNS,0,1761,17,False,3,NaN,2282,105439740,37.094768,-85.897407,2020-03-06 14:00:21.000
1187331,1248455,123905139,2020-03-06 14:04:23,Condition Exists Engine Protection Torque Derate,NaN,04358814*06099720*030816202706*09400153*G1*BDR*,79932020,6X1u13D1500000000,CMMNS,0,1569,31,True,5,NaN,1994,105354084,34.390740,-79.461805,2020-03-06 14:04:59.000
1187332,1248456,123905996,2020-03-06 14:13:38,Abnormal Rate of Change Aftertreatment 1 Intak...,NaN,05317106*05100987*050719120655*09401585*G1*BDR*,79880653,6X1u13D1500000000,CMMNS,0,3216,10,True,1,NaN,1850,105336308,34.430370,-84.920509,2020-03-06 14:14:14.000
1187333,1248457,123906113,2020-03-06 14:14:13,Low (Severity Medium) Engine Coolant Level,NaN,04384413*22544852*090619141107*60701756*G1*BGT*,NaN,NaN,NaN,0,111,18,True,8,NaN,2377,108605700,35.030925,-85.321527,2020-03-06 14:14:49.000


In [23]:
faults = faults.drop(columns = ["actionDescription", "faultValue"])

In [24]:
faults.shape

(1185166, 18)

In [25]:
faults.isna().sum()

RecordID                      0
ESS_Id                        0
EventTimeStamp                0
eventDescription          60366
ecuSoftwareVersion       295827
ecuSerialNumber          342772
ecuModel                  64649
ecuMake                   64649
ecuSource                     0
spn                           0
fmi                           0
active                        0
activeTransitionCount         0
EquipmentID                   0
MCTNumber                     0
Latitude                      0
Longitude                     0
LocationTimeStamp             0
dtype: int64

Remove faults occurring in the vicinity of the service locations at (36.0666667, -86.4347222), (35.5883333, -86.4438888), and (36.1950, -83.174722)

In [26]:
faults = faults[(faults['Latitude'] != '36.0666667') & (faults['Longitude'] != '-86.4347222')]
faults = faults[(faults['Latitude'] != '35.5883333') & (faults['Longitude'] != '-86.4438888')]
faults = faults[(faults['Latitude'] != '36.1950') & (faults['Longitude'] != '-83.174722')]

In [30]:
# create a sample GeoDataFrame
gdf = gpd.GeoDataFrame(faults, geometry=gpd.points_from_xy(faults['Longitude'], faults['Latitude']))

# calculate the radius of each point in meters
gdf['Radius'] = gdf['geometry'].apply(lambda x: distance(x, gdf['geometry'].iloc[0]))

TypeError: Failed to create Point instance from <POINT (-84.627 38.858)>.

In [28]:
faults.loc[faults['spn'] == 5246]

,RecordID,ESS_Id,EventTimeStamp,eventDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp,geometry
45,46,990931,2015-02-21 12:10:51,NaN,04993120*00027849*082113134117*07700053*I0*BBZ*,79464664,6X1u10D1500000000,CMMNS,0,5246,0,True,1,1395,105349612,36.065972,-86.433425,2015-02-21 12:11:27.000,POINT (-86.43342 36.06597)
1918,1919,1007751,2015-02-22 19:44:55,NaN,04993120*00027849*082113134117*07700053*I0*BBZ*,79464664,6X1u10D1500000000,CMMNS,0,5246,0,True,1,1395,105349612,36.066203,-86.434814,2015-02-22 19:46:27.000,POINT (-86.43481 36.06620)
2058,2059,1010486,2015-02-23 04:00:21,NaN,04993120*00027849*082113134117*07700053*I0*BBZ*,79464664,6X1u10D1500000000,CMMNS,0,5246,0,False,1,1395,105349612,36.066666,-86.434537,2015-02-23 01:06:06.000,POINT (-86.43454 36.06667)
2089,2090,1011009,2015-02-23 05:05:44,NaN,05290170*03015749*051914190353*09400015*G1*BDR*,79642446,6X1u13D1500000000,CMMNS,0,5246,0,True,1,1630,105329900,40.733009,-74.087777,2015-02-23 05:08:23.000,POINT (-74.08778 40.73301)
2971,2972,1026305,2015-02-23 15:54:22,NaN,unknown,unknown,unknown,unknown,0,5246,0,True,1,1487,105369355,28.077361,-81.897083,2015-02-23 15:54:58.000,POINT (-81.89708 28.07736)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1183032,1244156,121610128,2020-02-19 07:02:33,NaN,05317106*05005224*051718172255*09401583*G1*BDR*,79845785,6X1u13D1500000000,CMMNS,0,5246,0,True,1,1814,105369518,36.067037,-86.434120,2020-02-19 07:03:09.000,POINT (-86.43412 36.06704)
1183684,1244808,121909497,2020-02-21 07:23:44,NaN,04384413*22246857*090619141107*60701756*G1*BGT*,80092582,6X1u17D1500000000,CMMNS,0,5246,16,True,1,2211,105329862,36.066296,-86.434305,2020-02-21 07:24:20.000,POINT (-86.43431 36.06630)
1184328,1245452,122305094,2020-02-24 15:28:05,NaN,04384413*22246857*090619141107*60701756*G1*BGT*,80092582,6X1u17D1500000000,CMMNS,0,5246,16,False,1,2211,105329862,36.066620,-86.434722,2020-02-24 15:28:01.000,POINT (-86.43472 36.06662)
1184330,1245454,122305096,2020-02-24 15:27:26,NaN,04384413*22246857*090619141107*60701756*G1*BGT*,80092582,6X1u17D1500000000,CMMNS,0,5246,0,True,1,2211,105329862,36.066620,-86.434722,2020-02-24 15:28:02.000,POINT (-86.43472 36.06662)


In [12]:
service_fault

,Published in CES 14602,Cummins Fault Code,Revision,PID,SID,MID,J1587 FMI,SPN,J1939 FMI,J2012 Pcode,Lamp Color,Lamp Device,Cummins Description,Algorithm Description
0,Y,111,167,Not Mapped,254,0,12,629,12,P0606,Red,Stop / Shutdown,Engine Control Module Critical Internal Failur...,Error internal to the ECM related to memory ha...
1,Y,112,167,Not Mapped,20,128,7,635,7,Not Mapped,Red,Stop / Shutdown,Engine Timing Actuator Driver Circuit - Mechan...,Mechanical failure in the engine timing actuat...
2,Y,113,167,Not Mapped,20,128,3,635,3,Not Mapped,Amber,Warning,Engine Timing Actuator Driver Circuit - Voltag...,High signal voltage detected at the engine tim...
3,Y,114,167,Not Mapped,20,128,4,635,4,Not Mapped,Amber,Warning,Engine Timing Actuator Driver Circuit - Voltag...,Low voltage detected at the engine timing actu...
4,Y,115,167,190,Not Mapped,Not Mapped,2,612,2,P0008,Red,Stop / Shutdown,Engine Magnetic Speed/Position Lost Both of Tw...,The ECM has detected that the primary and back...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7119,Y,9996,167,Not Mapped,155,0,11,524286,31,Not Mapped,Amber,Warning,Reserved for temporary use - Condition Exists,NaN
7120,Y,9997,167,Not Mapped,155,0,11,524286,31,Not Mapped,Amber,Warning,Reserved for temporary use - Condition Exists,NaN
7121,Y,9998,167,Not Mapped,155,0,11,524286,31,Not Mapped,Amber,Warning,Reserved for temporary use - Condition Exists,NaN
7122,Y,9999,167,Not Mapped,155,0,11,524286,31,Not Mapped,Amber,Warning,Reserved for temporary use - Condition Exists,NaN


In [13]:
service_fault.shape

(7124, 14)

In [14]:
service_fault.isna().sum()

Published in CES 14602       0
Cummins Fault Code           0
Revision                     0
PID                          0
SID                          0
MID                          0
J1587 FMI                    0
SPN                          0
J1939 FMI                    0
J2012 Pcode                  0
Lamp Color                   0
Lamp Device                  0
Cummins Description          0
Algorithm Description     5119
dtype: int64

In [15]:
vehicle_Diagnostics

,Id,Name,Value,FaultId
0,1,IgnStatus,False,1
1,2,EngineOilPressure,0,1
2,3,EngineOilTemperature,96.74375,1
3,4,TurboBoostPressure,0,1
4,5,EngineLoad,11,1
...,...,...,...,...
12821621,12864020,EngineCoolantTemperature,181.4,1248457
12821622,12864021,ParkingBrake,False,1248457
12821623,12864022,SwitchedBatteryVoltage,14.1,1248457
12821624,12864023,DistanceLtd,28606.65625,1248457


In [16]:
vehicle_Diagnostics.shape

(12821626, 4)

In [17]:
vehicle_Diagnostics.isna().sum()

Id         0
Name       0
Value      0
FaultId    0
dtype: int64